# Demo

Make a mouse

Imports

In [1]:
from Management.Organization import Mouse

Here we create a structure for organizing the data & created an auto-logger

In [2]:
MyMouse = Mouse(Directory="D:\\Theodore", Mouse="Theodore")

Log file can only be assigned ONCE.
Experimental condition can only be assigned ONCE.
Logging file assigned as :D:\Theodore\log_file.log
Saving mouse...
Finished saving mouse.
Activating auto-logging. Current session state plus future input saved.
Filename       : D:\Theodore\log_file.log
Mode           : append
Output logging : True
Raw input log  : True
Timestamping   : True
State          : active
Logging Initiated


We can assign other identifiers during or after creation too

In [3]:
MyMouse.experimental_condition = "Gentlemen"
MyMouse.study = "Mouse Manners"
MyMouse.study_mouse = "G1"

We create folders for organizing the data files

In [4]:
MyMouse.create() # This makes a folder for any lab notebook / record files, the directory itself, and the organization.json for the mouse

Saving mouse...
Finished saving mouse.


In [5]:
MyMouse.create_experiment("Generic_Experiment_1", interactive=False) # Experiment is a default "super-class" for experimental sessions and is picked here by default. You can make classes that inherit all its functions and add others for specific types of experiments like "FearConditioning") #By default this creates a folder with sub-folders for behavioral data, imaging data, computational data, and figures

Imaging Experiment

In [6]:
from Management.Organization import ImagingExperiment

In [7]:
MyMouse.create_experiment("Imaging_Experiment_2", Type=ImagingExperiment, interactive=True)

Copying files...: 100%|██████████| 6616/6616 [00:17<00:00, 368.42it/s]


Behavioral Experiment

In [8]:
from Management.Organization import BehavioralExperiment

In [9]:
MyMouse.create_experiment("Behavioral_Experiment_3", Type=BehavioralExperiment, interactive=False) # When interactive is true, a tool is opened to copy the folders & automatically reorganize the files

Imaging/Behavioral Experiment

In [10]:
from Management.Organization import ImagingBehaviorExperiment

In [11]:
MyMouse.create_experiment("ImagingBehavior_Experiment_4", Type=ImagingBehaviorExperiment, interactive=False)

In [12]:
MyMouse.update_all_folder_dictionaries() # this function updates all the indexing of all folders and should probably be called before periodically or after changes. It's sometimes called as part of pertinent functions, but not inserted in all yet.

Updating Folder Dictionaries: 100%|██████████| 4/4 [00:01<00:00,  2.28it/s]


Save

In [13]:
MyMouse.save()

Saving mouse...
Finished saving mouse.


## Importing Imaging Files

Imports

In [14]:
from Imaging.IO import repackage_bruker_tiffs, load_all_tiffs, save_raw_binary

In [15]:
MyMouse.Imaging_Experiment_2.add_image_sampling_folder(6) # Make 30 Hz Folder

Compile into tiff stacks -->

In [16]:
repackage_bruker_tiffs(MyMouse.Imaging_Experiment_2.folder_dictionary.get("raw_imaging_data").path,
                       MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").folders.get("compiled")
                       )
# Input folder is raw imaging data, Output folder is the compiled folder in 6Hz folder



+-----------------------+------+
| Total Images Detected | 6609 |
|        Channels       |  1   |
|         Planes        |  1   |
|         Frames        | 6609 |
|         Height        | 256  |
|         Width         | 256  |
+-----------------------+------+


Repackaging Bruker Tiffs...: 100%|██████████| 6609/6609 [00:13<00:00, 507.99it/s]


Load the tiff stacks -->

In [17]:
images = load_all_tiffs(MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").folders.get("compiled"))

Loading Images...:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Do something here like down-sample or smooth if desired or preview the mean image

Save to raw binary -->

In [18]:
save_raw_binary(images, MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").folders.get("compiled")) # save it as raw binary as its more advantageous
del images # delete from memory

Saving images as a binary file...
Finished saving images as a binary file.


Clean-up compiled tiff stacks since we don't need them and they take up space on the disk. We could alternatively have just loaded the bruker tiffs and then saved straight to raw binary, but i find the repackaging takes a while and its worth saving to tiff if I want to preview/down-sample/etc

In [19]:
MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").clean_up_compilation() # Delete the old now binary files to save storage space / etc

In [20]:
MyMouse.update_all_folder_dictionaries() # Update folders to be sure

Updating Folder Dictionaries: 100%|██████████| 4/4 [00:01<00:00,  2.44it/s]


In [21]:
MyMouse.save()

Saving mouse...
Finished saving mouse.


Here are the steps for imaging analysis

In [22]:
from Management.Organization import ImagingAnalysis
ImagingAnalysis.current_experiment_step?

## Run Suite2P for motion correction and ROI identification

Imports

In [25]:
from Imaging.ToolWrappers.Suite2PModule import Suite2PAnalysis
import numpy as np

In [26]:
S2P = Suite2PAnalysis(MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").folders.get("compiled"), MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").path, file_type="binary")
# Input folder is the compiled folder
# Output folder is suite2p folder
# file type is binary

Warning Folder Already Exists and Will be Overwritten


NameError: name 'ExperimentData' is not defined

In [29]:
S2P.motionCorrect() # Run Motion Correction
MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").export_registration_to_denoised()
S2P.ops["meanImg_chan2"] = np.array([0]) # Don't question, needed for now
S2P.ops.pop("meanImg_chan2") # Don't question, needed for now
S2P.db  = S2P.ops # Don't question, needed for now
S2P.roiDetection()
S2P.extractTraces()
S2P.classifyROIs()
S2P.spikeExtraction()
S2P.iscell, S2P.stat = S2P.remove_small_neurons(S2P.iscell, S2P.stat)
S2P.save_files()
MyMouse.record_stage_mod('Imaging_Experiment_2', 'We ran suite2p on behavior session 1')
MyMouse.update_all_folder_dictionaries()
MyMouse.save_experiments()

Reference frame, 26.83 sec.
Registered 887/887 in 5.96s
Saving images as a binary file...
Finished saving images as a binary file.
Binning movie in chunks of length 15
Binned movie of size [59,510,510] created in 0.66 sec.
Binned movie denoised (for cell detection only) in 3.71 sec.
>>>> CELLPOSE finding masks in max_proj / mean_img
!NOTE! diameter set to 14.00 for cell detection with cellpose
>>>> 10 masks detected, median diameter = 12.10 
Detected 10 ROIs, 6.41 sec
After removing overlaps, 10 ROIs remain
Masks created, 0.42 sec.
Extracted fluorescence from 10 ROIs in 887 frames, 5.52 sec.
['compact', 'npix_norm', 'skew']


Updating Folder Dictionaries: 100%|██████████| 1/1 [00:00<00:00,  4.54it/s]

Saving Experiments...
Finished.


In [39]:
S2P.openGUI()

NameError: name 'S2P' is not defined

## Fissa Source Separation

In [36]:
from Imaging.ToolWrappers.FissaModule import FissaAnalysis

In [38]:
MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").clean_up_motion_correction()
Fissa = FissaAnalysis(data_folder=MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").path, video_folder=MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").folders.get("denoised"), output_folder=MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").folders.get("fissa"))
Fissa.initializeFissa()
Fissa.extractTraces() # simple, call to extract raw traces from videos
Fissa.saveFissaPrep()
Fissa.passPrepToFissa()
Fissa.separateTraces() # simple, call to separate the traces
Fissa.saveFissaSep()


Loading and Splitting Images



TypeError: unsupported operand type(s) for /: 'int' and 'NoneType'

In [ ]:
from Imaging.SignalProcessing import smoothTraces_TiffOrg, detrendTraces, calculate_dFoF
from Imaging.Utilities import mergeTraces

In [ ]:
Fissa.ProcessedTraces.smoothed_result = smoothTraces_TiffOrg(Fissa.experiment.result, niter=25, kappa=0.15, gamma=0.25)[0]

# Calculate Fo/F
Fissa.ProcessedTraces.dFoF_result = calculate_dFoF(Fissa.ProcessedTraces.smoothed_result, Fissa.frame_rate, raw=Fissa.preparation.raw, merge_after=False)

# Condense the ROI Traces for each Trial into a Single Matrix
Fissa.ProcessedTraces.merged_dFoF_result = mergeTraces(Fissa.ProcessedTraces.dFoF_result)

for _trace in range(Fissa.ProcessedTraces.merged_dFoF_result.shape[0]):
    Fissa.ProcessedTraces.merged_dFoF_result[_trace, :] = Fissa.ProcessedTraces.merged_dFoF_result[_trace, :] - np.mean(Fissa.ProcessedTraces.merged_dFoF_result[_trace, :])

# Detrend the Traces by fitting a 4th-order polynomial and subsequently subtracting
Fissa.ProcessedTraces.detrended_merged_dFoF_result = detrendTraces(Fissa.ProcessedTraces.merged_dFoF_result, order=4, plot=False)

Fissa.saveProcessedTraces()
MyMouse.record_stage_mod("Imaging_Experiment_2", "Fissa")
MyMouse.save_experiments()

## Cascade Spike Inference

In [ ]:
from Imaging.ToolWrappers.CascadeModule import CascadeAnalysis

In [ ]:
Cascade = CascadeAnalysis(Fissa.ProcessedTraces.detrended_merged_dFoF_result, 30, model_folder="C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\Pretrained_models", SavePath=MyMouse.Imaging_Experiment_2.folder_dictionary.get("imaging_6Hz").folders.get("cascade"))

In [ ]:
# Pull Available Models
list_of_models = Cascade.pullModels(Cascade.model_folder)
# Select Model: If you know what model you want, you should use the string instead.
# This model is Global_EXC_10Hz_smoothing_100ms
# Cascade.model_name = list_of_models[21]
Cascade.model_name = "Global_EXC_30Hz_smoothing100ms"
Cascade.downloadModel(Cascade.model_name, "C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\Pretrained_models")
Cascade.predictSpikeProb() # Simple, call to infer spike probability for each frame
# Calculate Firing Rates # Simple, firing rate = spike probability * imaging frequency
Cascade.ProcessedInferences.firing_rates = calculateFiringRate(Cascade.spike_prob, Cascade.frame_rate)
Cascade.saveSpikeProb()
Cascade.saveProcessedInferences()
Cascade.inferDiscreteSpikes()
Cascade.saveSpikeInference()
MyMouse.record_experiment_mod("Imaging_Experiment_2", "Cascade")
MyMouse.save()